The following file is for Assignment 2 for the Course "Introduction to Data Science" from Michigan University on the Coursera platform

In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)
data = pd.read_csv("datasets/NISPUF17.csv",index_col=0)
data.head()

,SEQNUMC,SEQNUMHH,PDAT,PROVWT_D,RDDWT_D,STRATUM,YEAR,AGECPOXR,HAD_CPOX,AGEGRP,...,XVRCTY2,XVRCTY3,XVRCTY4,XVRCTY5,XVRCTY6,XVRCTY7,XVRCTY8,XVRCTY9,INS_STAT2_I,INS_BREAK_I
1,128521,12852,2,NaN,235.916956,1031,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10741,1074,2,NaN,957.353840,1068,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,220011,22001,2,NaN,189.611299,1050,2017,NaN,2,3,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86131,8613,1,675.430817,333.447418,1040,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
5,227141,22714,1,482.617748,278.768063,1008,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0


Question 1: Write a function called proportion_of_education which returns the proportion of children in the dataset who had a mother with the education levels equal to less than high school (<12), high school (12), more than high school but not a college graduate (>12) and college degree.

This function should return a dictionary in the form of (use the correct numbers, do not round numbers):

    {"less than high school":0.2,
    "high school":0.4,
    "more than high school but not college":0.2,
    "college":0.2}

In [19]:
# Extract the maternal education
momedu = data['EDUC1']

# Calculate the proportion of each level of maternal education
less_than_hs = (momedu == 1).mean()
equal_hs = (momedu == 2).mean()
more_than_hs = (momedu == 3).mean()
college = (momedu == 4).mean()

# Create a dictionary to store the proportions
proportion_of_education = {'less than high school': less_than_hs,
                            'high school':equal_hs,
                            'more than high school but not college':more_than_hs,
                            'college':college}

# Print the dictionary
print(proportion_of_education)


{'less than high school': 0.10202002459160373, 'high school': 0.172352011241876, 'more than high school but not college': 0.24588090637625154, 'college': 0.47974705779026877}


Question 2: Let's explore the relationship between being fed breastmilk as a child and getting a seasonal influenza vaccine from a healthcare provider. Return a tuple of the average number of influenza vaccines for those children we know received breastmilk as a child and those who know did not.

This function should return a tuple in the form (use the correct numbers:

(2.5, 0.1)


In [23]:
#first 'CBF_01' is the colum child ever fed breast milk and second,'P_NUMFLU' the total number of seasonal influenza doses

milk_and_vac = data.loc[data["CBF_01"] == 1, "P_NUMFLU"].mean() #children fed with milk and number of vaccines
not_milk_and_vac = data.loc[data["CBF_01"] == 2,"P_NUMFLU"].mean() #children not fed with milk and number of vaccines

print(milk_and_vac,not_milk_and_vac)

1.8799187420058687 1.5963945918878317


Question 3:It would be interesting to see if there is any evidence of a link between vaccine effectiveness and sex of the child. Calculate the ratio of the number of children who contracted chickenpox but were vaccinated against it (at least one varicella dose) versus those who were vaccinated but did not contract chicken pox. Return results by sex.

This function should return a dictionary in the form of (use the correct numbers):

    {"male":0.2,
    "female":0.4}
Note: To aid in verification, the chickenpox_by_sex()['female'] value the autograder is looking for starts with the digits 0.0077.

In [30]:
# Filter data only for children who have received the vaccine againts varicella at least one time
chicken = data.loc[data['P_NUMVRC'] >= 1, ['SEX', 'HAD_CPOX']]

# Count the number of male and female children who received the chickenpox vaccine and had chickenpox
chicken_male = chicken.loc[chicken['SEX'] == 1, 'HAD_CPOX'].value_counts()
#print(chicken_male) 
chicken_female = chicken.loc[chicken['SEX'] == 2, 'HAD_CPOX'].value_counts()
#print(chicken_female)

# Calculate the ratio of male to female children who received the chickenpox vaccine and had chickenpox
ratio_male = chicken_male[1] / chicken_male[2]
ratio_female = chicken_female[1] / chicken_female[2]

{'male':ratio_male,'female':ratio_female}


{'male': 0.009675583380762664, 'female': 0.0077918259335489565}

Question 4: A correlation is a statistical relationship between two variables. If we wanted to know if vaccines work, we might look at the correlation between the use of the vaccine and whether it results in prevention of the infection or disease [1]. In this question, you are to see if there is a correlation between having had the chicken pox and the number of chickenpox vaccine doses given (varicella).

Some notes on interpreting the answer. The had_chickenpox_column is either 1 (for yes) or 2 (for no), and the num_chickenpox_vaccine_column is the number of doses a child has been given of the varicella vaccine. A positive correlation (e.g., corr > 0) means that an increase in had_chickenpox_column (which means more no’s) would also increase the values of num_chickenpox_vaccine_column (which means more doses of vaccine). If there is a negative correlation (e.g., corr < 0), it indicates that having had chickenpox is related to an increase in the number of vaccine doses.

Also, pval is the probability that we observe a correlation between had_chickenpox_column and num_chickenpox_vaccine_column which is greater than or equal to a particular value occurred by chance. A small pval means that the observed correlation is highly unlikely to occur by chance. In this case, pval should be very small (will end in e-18 indicating a very small number).

[1] This isn’t really the full picture, since we are not looking at when the dose was given. It’s possible that children had chickenpox and then their parents went to get them the vaccine. Does this dataset have the data we would need to investigate the timing of the dose?

In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd
    
      
# this is just an example dataframe
df=pd.DataFrame({"had_chickenpox_column":np.random.randint(1,3,size=(100)),
              "num_chickenpox_vaccine_column":np.random.randint(0,6,size=(100))})

# here is some stub code to actually run the correlation
corr, pval=stats.pearsonr(df["had_chickenpox_column"],df["num_chickenpox_vaccine_column"])
    

df = pd.read_csv("datasets/NISPUF17.csv", index_col=0)

df=df[df['HAD_CPOX'].lt(3)].loc[:,['HAD_CPOX','P_NUMVRC']].dropna()
df.columns=['had_chickenpox_column','num_chickenpox_vaccine_column']
# here is some stub code to actually run the correlation
corr, pval=stats.pearsonr(df["had_chickenpox_column"],df["num_chickenpox_vaccine_column"])
    
# just return the correlation
print(corr)

0.07044873460147864
